In [17]:
import json
import os
import pandas as pd

top_p = 0.9

max_new_tokens=100

num_completions=200

make_sense = [True,False]

human_labeled = [True,False]

domains = ["beatnum","monkey","numpy","pandas","torchdata"]

api_num = ["0","1","2","3","5","n"]

temperature = [0.1,0.3,0.5,0.7,0.9,1.0]

base_dir = "/Users/yuchientsai/Documents/code_gen/PyCodeGPT/apicoder/data/eval_result/gpt"

result_df = pd.DataFrame(columns=["domain","api_num","temperature","make_sense","pass@1","pass@10","pass@100"])
already_add =[]
for ms in make_sense:
    for hl in human_labeled:
        for domain in domains:
            for apin in api_num:
                for temp in temperature:
                    
                    if hl:
                        if ms:
                            output_file_name = f"real_{domain}_eval_v3_human_labelled_make_sense.API_Coder.hm_True.human_labelled.t{temp}.p{top_p}.l{max_new_tokens}.n{num_completions}.samples.jsonl_metrics.jsonl"
                        elif not ms:
                            output_file_name = f"real_{domain}_eval_v3_human_labelled.API_Coder.hm_True.human_labelled.t{temp}.p{top_p}.l{max_new_tokens}.n{num_completions}.samples.jsonl_metrics.jsonl"
                    else:
                        if apin in ["1","2","3","5","n"] :
                            if ms:
                                output_file_name = f"real_{domain}_eval_v3_api_{apin}_make_sense.API_Coder.hm_False.machine.t{temp}.p{top_p}.l{max_new_tokens}.n{num_completions}.samples.jsonl_metrics.jsonl"
                            elif not ms:
                                output_file_name = f"real_{domain}_eval_v3_api_{apin}.API_Coder.hm_False.machine.t{temp}.p{top_p}.l{max_new_tokens}.n{num_completions}.samples.jsonl_metrics.jsonl"

                        elif apin in ["0"] :
                            if ms:
                                output_file_name = f"real_{domain}_eval_v3_make_sense.API_Coder.hm_False.machine.t{temp}.p{top_p}.l{max_new_tokens}.n{num_completions}.samples.jsonl_metrics.jsonl"
                            elif not ms:
                                output_file_name = f"real_{domain}_eval_v3.API_Coder.hm_False.machine.t{temp}.p{top_p}.l{max_new_tokens}.n{num_completions}.samples.jsonl_metrics.jsonl"
                    #print(output_file_name)          
                    metric_file_path = f"{base_dir}/{output_file_name}"
                    if os.path.exists(metric_file_path) and (metric_file_path not in already_add):
                        already_add.append(metric_file_path)
                        with open(metric_file_path,"r") as f:
                            metric = json.load(f)
                            if hl:
                                apin = "human_labelled"
                            
                            data = {"domain":domain,
                                    "api_num": apin,
                                    "temperature":temp,
                                    "make_sense":ms,
                                    "pass@1":metric["pass@1"],
                                    "pass@10":metric["pass@10"],
                                    "pass@100":metric["pass@100"],
                                   }
                            #print(data)
                            #result_df = result_df.append(new_data, ignore_index=True)
                            result_df = pd.concat([result_df, pd.DataFrame([data])], ignore_index=True)

                        




KeyboardInterrupt: 

In [ ]:
domain = "monkey"
print(result_df[(result_df["domain"] == domain) * (result_df["api_num"] == "0")].max(),"\n"+"="*100)
print(result_df[(result_df["domain"] == domain) * (result_df["api_num"] == "1")].max(),"\n"+"="*100)
print(result_df[(result_df["domain"] == domain) * (result_df["api_num"] == "2")].max(),"\n"+"="*100)
print(result_df[(result_df["domain"] == domain) * (result_df["api_num"] == "3")].max(),"\n"+"="*100)
print(result_df[(result_df["domain"] == domain) * (result_df["api_num"] == "5")].max(),"\n"+"="*100)
print(result_df[(result_df["domain"] == domain) * (result_df["api_num"] == "n")].max(),"\n"+"="*100)
print(result_df[(result_df["domain"] == domain) * (result_df["api_num"] == "human_labelled")].max(),"\n"+"="*100)

In [ ]:
#result_df.to_excel('./result.xlsx', index=False)

In [18]:
import ipdb
import gzip
import json
from typing import Iterable, Dict

def stream_jsonl(filename: str) -> Iterable[Dict]:
    """
    Parses each jsonl line and yields it as a dictionary
    """
    if filename.endswith(".gz"):
        with open(filename, "rb") as gzfp:
            with gzip.open(gzfp, 'rt') as fp:
                for line in fp:
                    if any(not x.isspace() for x in line):
                        yield json.loads(line)
    else:
        with open(filename, "r") as fp:
            for line in fp:
                if any(not x.isspace() for x in line):
                    try:
                        yield json.loads(line)
                    except:
                        ipdb.set_trace()

In [28]:
import os
from glob import glob
base_dir = "/Users/yuchientsai/Documents/code_gen/PyCodeGPT/apicoder/data/eval_result/gpt"
problem_dir = "/Users/yuchientsai/Documents/code_gen/PyCodeGPT/apicoder/private-eval/data"


gpt3_gen_result_paths = sorted(glob(f"{base_dir}/real_torchdata*l500.n1.gen_code.jsonl"))
#print(gpt3_gen_result_paths)

for path in gpt3_gen_result_paths:
    out_path = path.replace(".gen_code",".gen_code.samples.test")
    
    problem_filename = f"{os.path.basename(path).split('.API_Coder')[0]}.jsonl.gz"
    problem_path = os.path.join(problem_dir, problem_filename)
    
    task_id2task = {}
    if os.path.exists(problem_path):
        for data in stream_jsonl(problem_path) :
            task_id2task[data["task_id"]] = data
    else:
        raise FileNotFoundError(problem_path)
    
    out_data = []
    with open(path,"r") as f :
        for line in f:
            
            datum = json.loads(line)
            print("="*50 + datum["task_id"] + "="*50)
            if datum["success"]:
                prompt = task_id2task[datum["task_id"]]["prompt"]
                gen_code = datum["generated_code"]
                
                prompt_splits = prompt.split("\n")
                while True:
                    prompt_last_line = prompt_splits.pop(-1)
                    if prompt_last_line.strip() == '\"\"\"':
                        print("prompt_last_line:",prompt_last_line)
                        prompt_last_line = prompt_splits.pop(-1) + '\n' + prompt_last_line
                        print("last_line:",'\"\"\"')
                        print("prompt_last_line:",prompt_last_line)
                        print("prompt:",prompt)
                        break
                    if len(prompt_last_line.strip()) > 0:
                        prompt_last_line = prompt_last_line.strip()
                        print("last_line:",prompt_last_line)
                        print("prompt_last_line:",prompt_last_line)
                        print("prompt:",prompt)
                        break
                    if len(prompt_splits) == 0:
                        prompt_last_line = "jfil;ajfl;l;ji;jfila;s"
                        break
                if " changged " in prompt_last_line:
                    prompt_last_line = prompt_last_line.replace(" changged "," changed ")
                prompt_last_line = prompt_last_line.strip().lstrip("#").strip()
                if prompt_last_line in gen_code:
                    completion_splits = gen_code.split(prompt_last_line)[1:]
                    if len(completion_splits) == 1:
                        completion = completion_splits[0]
                    elif len(completion_splits) > 1:
                        completion = prompt_last_line.join(completion_splits)
                    else:
                        completion = gen_code
                else:
                    print("last line not in gen code")
                    completion = gen_code
                completion_splits = completion.split("\n")
                first_list = 0
                for split in completion_splits:
                    if (split.strip() != '\"\"\"') and len(split.strip())>0:
                        break
                    else:
                        first_list +=1
                if prompt_last_line.strip() == "return" or prompt_last_line.strip().endswith("="):
                    completion = "\n".join(completion_splits[first_list:])
                else:
                    completion = "\n" + "\n".join(completion_splits[first_list:])
                        
                print("gen_code:",gen_code)
                print("completion:",completion)
                out_data.append({"task_id": datum["task_id"],
                                "completion": completion})
            else:
                out_data.append({"task_id": datum["task_id"],
                                "completion": ""})
            print("="*100)
    with open(out_path,"w") as out_f:
        for datum in out_data:
            out_f.write(json.dumps(datum) + "\n")

==================================================TorchDataEval/0==================================================
last_line: new_datapipe =
prompt_last_line: new_datapipe =
prompt: from torchdata.datapipes.iter import IterableWrapper
datapipe = IterableWrapper([1,2,3])
# How to augument the datapipe by repeating it six times.
new_datapipe =
gen_code: from torchdata.datapipes.iter import IterableWrapper

datapipe = IterableWrapper([1,2,3])
# How to augment the datapipe by repeating it six times.
new_datapipe = datapipe.repeat(6)
completion:  datapipe.repeat(6)
==================================================TorchDataEval/1==================================================
last_line: new_dp =
prompt_last_line: new_dp =
prompt: from torchdata.datapipes.iter import IterableWrapper

dp = IterableWrapper(['a', 'b', 'c'])
# Assign indexs to the datepipe object.
new_dp =
gen_code: from torchdata.datapipes.iter import IterableWrapper

dp = IterableWrapper(['a', 'b', 'c'])
# Assign indexs to

In [ ]:
import json
import os
import pandas as pd

top_p = 0.9

max_new_tokens=500

num_completions=1

make_sense = [True,False]

human_labeled = [True,False]

domains = ["beatnum","monkey","numpy","pandas","torchdata"]

api_num = ["0","1","2","3","5","n"]

temperature = [0.1,0.3,0.5,0.7,0.9,1.0]

base_dir = "/share/cyq/code_gen_experiment/PyCodeGPT/apicoder/data/CodeGenAPI/CodeGenAPI-350M-mono"

result_df = pd.DataFrame(columns=["domain","api_num","temperature","make_sense","pass@1"])
already_add =[]
for ms in make_sense:
    for hl in human_labeled:
        for domain in domains:
            for apin in api_num:
                for temp in temperature:
                    
                    if hl:
                        if ms:
                            output_file_name = f"real_{domain}_eval_v3_human_labelled_make_sense.API_Coder.hm_True.human_labelled.t{temp}.p{top_p}.l{max_new_tokens}.n{num_completions}.gen_code.samples.jsonl_metrics.jsonl"
                        elif not ms:
                            output_file_name = f"real_{domain}_eval_v3_human_labelled.API_Coder.hm_True.human_labelled.t{temp}.p{top_p}.l{max_new_tokens}.n{num_completions}.gen_code.samples.jsonl_metrics.jsonl"
                    else:
                        if apin in ["1","2","3","5","n"] :
                            if ms:
                                output_file_name = f"real_{domain}_eval_v3_api_{apin}_make_sense.API_Coder.hm_False.machine.t{temp}.p{top_p}.l{max_new_tokens}.n{num_completions}.gen_code.samples.jsonl_metrics.jsonl"
                            elif not ms:
                                output_file_name = f"real_{domain}_eval_v3_api_{apin}.API_Coder.hm_False.machine.t{temp}.p{top_p}.l{max_new_tokens}.n{num_completions}.gen_code.samples.jsonl_metrics.jsonl"

                        elif apin in ["0"] :
                            if ms:
                                output_file_name = f"real_{domain}_eval_v3_make_sense.API_Coder.hm_False.machine.t{temp}.p{top_p}.l{max_new_tokens}.n{num_completions}.gen_code.samples.jsonl_metrics.jsonl"
                            elif not ms:
                                output_file_name = f"real_{domain}_eval_v3.API_Coder.hm_False.machine.t{temp}.p{top_p}.l{max_new_tokens}.n{num_completions}.gen_code.samples.jsonl_metrics.jsonl"
                    #print(output_file_name)          
                    metric_file_path = f"{base_dir}/{output_file_name}"
                    if os.path.exists(metric_file_path) and (metric_file_path not in already_add):
                        already_add.append(metric_file_path)
                        with open(metric_file_path,"r") as f:
                            metric = json.load(f)
                            if hl:
                                apin = "human_labelled"
                            
                            data = {"domain":domain,
                                    "api_num": apin,
                                    "temperature":temp,
                                    "make_sense":ms,
                                    "pass@1":metric["pass@1"],
                                   }
                            #print(data)
                            #result_df = result_df.append(new_data, ignore_index=True)
                            result_df = pd.concat([result_df, pd.DataFrame([data])], ignore_index=True)



In [ ]:
result_df.to_excel('./gpt35_result.xlsx', index=False)